In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import pandas as pd

In [3]:
df=pd.read_excel('./빌보드.xlsx')
len(df)

3596

In [4]:
#노래 제목에 *부분이 있는 음악은 검색이 불가능하여 제거
cond1=df['제목'].str.contains('\*')==False
cond=cond1
df=df[cond]
len(df)

3596

In [11]:
browser = webdriver.Chrome()
url='https://www.genie.co.kr/'
browser.get(url)
time.sleep(1)

In [9]:
result=[]

In [10]:
result

[]

In [12]:
for i in tqdm(range(len(df))):
    search=browser.find_elements('css selector', '#sc-fd')
    title=df.iloc[i][3]
    siger=df.iloc[i][4]
    search[0].click()
    search[0].clear()
    search[0].send_keys(f'{siger} {title}\n')
    time.sleep(1)
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    #검색한 노래가 있는지 숫자로 확인(0이면 없다는 의미이다)
    num=soup.select('div.search_song > div > span.article')
    
    #검색한 노래가 있는 경우
    if len(num)!=0:
        place=browser.find_elements('css selector', 'td.link > a')[0]
        place.click()
        time.sleep(1)
        html = browser.page_source
        soup = BeautifulSoup(html, 'html.parser')
        information=soup.select('ul.info-data > li > span.value')
        mood=information[2].text.strip()
        music_length=information[3].text.strip()
        lyrics_list=soup.select('pre#pLyrics > p')
        lyrics=lyrics_list[0].text.strip()
        data=mood,music_length,lyrics
        result.append(data)
    
    #검색한 노래가 없는 경우
    else:
        
        #검색한 노래가 없는 경우중에 가수이름에 Featuring가 들어가 있는 경우 Featuring앞의 내용만 출력
        if 'Featuring' in siger:
            split_sentence = siger.split("Featuring")
            siger=split_sentence[0].strip()
            search=browser.find_elements('css selector', '#sc-fd')
            search[0].click()
            search[0].clear()
            search[0].send_keys(f'{siger} {title}\n')
            time.sleep(1)
            html = browser.page_source
            soup = BeautifulSoup(html, 'html.parser')
            
            #Featuring앞의 내용의 가수로 다시 검색
            num=soup.select('div.search_song > div > span.article')
            
            #노래가 존재하는 경우
            if(len(num)!=0):
                place=browser.find_elements('css selector', 'td.link > a')[0]
                place.click()
                time.sleep(1)
                html = browser.page_source
                soup = BeautifulSoup(html, 'html.parser')
                information=soup.select('ul.info-data > li > span.value')
                mood=information[2].text.strip()
                music_length=information[3].text.strip()
                lyrics_list=soup.select('pre#pLyrics > p')
                lyrics=lyrics_list[0].text.strip()
                data=mood,music_length,lyrics
                result.append(data)
                
            #검색한 노래가 없는경우 미정으로 처리
            else:
                mood='미정'
                music_length='미정'
                lyrics='미정'
                data=mood,music_length,lyrics
                result.append(data)
        
        #Featuring가 포함되지 않은 경우
        else:
            
            #처음 두 부분의 이름을 검색하여 노래를 찾는다.
            split=siger.split()
            siger=' '.join(split[:2])
            search=browser.find_elements('css selector', '#sc-fd')
            search[0].click()
            search[0].clear()
            search[0].send_keys(f'{siger} {title}\n')
            time.sleep(1)
            html = browser.page_source
            soup = BeautifulSoup(html, 'html.parser')
            num=soup.select('div.search_song > div > span.article')
            
            #노래가 있는 경우
            if(len(num)!=0):
                place=browser.find_elements('css selector', 'td.link > a')[0]
                place.click()
                time.sleep(1)
                html = browser.page_source
                soup = BeautifulSoup(html, 'html.parser')
                information=soup.select('ul.info-data > li > span.value')
                mood=information[2].text.strip()
                music_length=information[3].text.strip()
                lyrics_list=soup.select('pre#pLyrics > p')
                lyrics=lyrics_list[0].text.strip()
                data=mood,music_length,lyrics
                result.append(data)
                
            #노래가 없는 경우 미정 처리
            else:
                mood='미정'
                music_length='미정'
                lyrics='미정'
                data=mood,music_length,lyrics
                result.append(data)

100%|████████████████████████████████████████████████████████████████████████████| 3596/3596 [3:57:20<00:00,  3.96s/it]


In [13]:
df = pd.DataFrame(result)
df.columns =['장르','재생시간','가사']

In [14]:
df.to_excel('./장르_가사.xlsx', index = False)